## Cifar-10 CNN
### Import, init

In [1]:
import keras
import numpy as np
import pickle

keras.backend.set_image_data_format('channels_first')
print('Using data format:', keras.backend.image_data_format())

Using TensorFlow backend.


Using data format: channels_first


### Read data

In [2]:
img_row, img_col, img_cnl = 32, 32, 3
label_names = { }
def read_imgs(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    imgs = np.array(data[b'data'])
    imgs = imgs.reshape((imgs.shape[0], img_cnl, img_row, img_col))
    labels = np.array(data[b'labels'])
    return imgs, labels
def load_labels(filename):
    f = open(filename, 'rb')
    data = pickle.load(f, encoding='bytes')
    f.close()
    return data[b'label_names']
print('# test: ')
%time imgs, labels = read_imgs('datas/data_batch_1')
print('# data shape:' ,imgs.shape)
print('# label names')
print(load_labels('datas/batches.meta'))

# test: 
Wall time: 48.3 ms
# data shape: (10000, 3, 32, 32)
# label names
[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']


read all data from file

In [3]:
# Load datas
train_x, train_y = read_imgs('datas/data_batch_1')
for x in (2, 3, 4, 5):
    fn = 'datas/data_batch_%d' % x
    imgs, labels = read_imgs(fn)
    train_x = np.concatenate((train_x, imgs))
    train_y = np.concatenate((train_y, labels))
test_x, test_y = read_imgs('datas/test_batch')

# Load label names
label_names = load_labels('datas/batches.meta')
label_names = [x.decode('ascii') for x in label_names]

# Categorical
class_cnt = len(label_names)
train_y = keras.utils.to_categorical(train_y, class_cnt)
test_y = keras.utils.to_categorical(test_y, class_cnt)

# Standard
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255

print('# Train data loaded:', train_x.shape[0])
print('# Test data loaded:', test_x.shape[0])
print('# Label names:')
print(label_names)

# Train data loaded: 50000
# Test data loaded: 10000
# Label names:
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## Define network structure
Conv -> Conv -> Pooling -> Conv -> Conv -> Pooling -> Dense

In [5]:
batch_size = 128
epochs = 200

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=train_x.shape[1:]))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(class_cnt, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train and evaluate

In [6]:
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_x, test_y))
model.evaluate(test_x, test_y, verbose=0)

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 39s - loss: 1.6358 - acc: 0.3974 - val_loss: 1.2714 - val_acc: 0.5455
Epoch 2/200
50000/50000 [==============================] - 37s - loss: 1.2128 - acc: 0.5687 - val_loss: 1.0620 - val_acc: 0.6287
Epoch 3/200
50000/50000 [==============================] - 37s - loss: 1.0157 - acc: 0.6411 - val_loss: 0.9031 - val_acc: 0.6810
Epoch 4/200
50000/50000 [==============================] - 37s - loss: 0.8976 - acc: 0.6837 - val_loss: 0.8035 - val_acc: 0.7158
Epoch 5/200
50000/50000 [==============================] - 37s - loss: 0.8237 - acc: 0.7097 - val_loss: 0.7998 - val_acc: 0.7207
Epoch 6/200
50000/50000 [==============================] - 37s - loss: 0.7657 - acc: 0.7313 - val_loss: 0.7254 - val_acc: 0.7527
Epoch 7/200
50000/50000 [==============================] - 37s - loss: 0.7125 - acc: 0.7502 - val_loss: 0.6912 - val_acc: 0.7624
Epoch 8/200
50000/50000 [======================

50000/50000 [==============================] - 36s - loss: 0.2312 - acc: 0.9190 - val_loss: 0.6624 - val_acc: 0.8117
Epoch 64/200
50000/50000 [==============================] - 36s - loss: 0.2195 - acc: 0.9228 - val_loss: 0.6954 - val_acc: 0.8093
Epoch 65/200
50000/50000 [==============================] - 36s - loss: 0.2280 - acc: 0.9217 - val_loss: 0.6674 - val_acc: 0.8140
Epoch 66/200
50000/50000 [==============================] - 36s - loss: 0.2205 - acc: 0.9221 - val_loss: 0.6877 - val_acc: 0.8072
Epoch 67/200
50000/50000 [==============================] - 37s - loss: 0.2258 - acc: 0.9204 - val_loss: 0.6631 - val_acc: 0.8101
Epoch 68/200
 8064/50000 [===>..........................] - ETA: 29s - loss: 0.1998 - acc: 0.9273

KeyboardInterrupt: 